# Scrooge's Bonus Helper Library

Core data structures and simulation utilities for the Scrooge's Bonus gift-exchange tournament. Import this notebook with `%run game_helpers.ipynb` inside other notebooks (e.g., `run_game.ipynb`) to access `run_match`, `run_multi_match`, and reporting helpers. Employer actions are now themed as "turkey" and "no_turkey".


In [ ]:
import pandas as pd
from dataclasses import dataclass
from typing import Callable, Dict, Mapping, Tuple

EmployerAction = str
EmployeeAction = str

VALID_EMPLOYER_ACTIONS = {"turkey", "no_turkey"}
VALID_EMPLOYEE_ACTIONS = {"high", "low"}

PAYOFF_MATRIX: Dict[Tuple[EmployerAction, EmployeeAction], Tuple[int, int]] = {
    ("turkey", "high"): (2, 2),
    ("turkey", "low"): (0, 3),
    ("no_turkey", "high"): (3, 0),
    ("no_turkey", "low"): (1, 1),
}

HISTORY_COLUMNS = ["employer_action", "my_action", "high_effort", "low_effort"]
EMPLOYER_HISTORY_COLUMNS = ["employer_action", "high_effort", "low_effort"]


In [ ]:
EmployerStrategy = Callable[[pd.DataFrame], EmployerAction]
EmployeeStrategy = Callable[[pd.DataFrame], EmployeeAction]


@dataclass
class MatchResult:
    history: pd.DataFrame
    employer_total: int
    employee_total: int

    def as_dict(self) -> Dict[str, int]:
        return {
            "employer_total": self.employer_total,
            "employee_total": self.employee_total,
        }


In [ ]:
def _validate_action(label: str, action: str, valid_set: set[str]) -> str:
    if action not in valid_set:
        allowed = ", ".join(sorted(valid_set))
        raise ValueError(f"{label} must be one of {allowed}, got '{action}'")
    return action


def _history_frame(records: list[dict], columns: list[str]) -> pd.DataFrame:
    if not records:
        return pd.DataFrame(columns=columns)
    df = pd.DataFrame(records).set_index("round")
    return df[columns]


def run_match(
    employer_fn: EmployerStrategy,
    employee_fn: EmployeeStrategy,
    rounds: int = 10,
) -> MatchResult:
    """Play repeated Scrooge's Bonus rounds (turkey/no_turkey) and return history + totals."""
    if rounds <= 0:
        raise ValueError("rounds must be > 0")

    history_records = []
    employer_history: list[dict] = []
    employee_history: list[dict] = []

    for round_idx in range(1, rounds + 1):
        employer_view = _history_frame(employer_history, EMPLOYER_HISTORY_COLUMNS)
        employee_view = _history_frame(employee_history, HISTORY_COLUMNS)

        employer_action = _validate_action(
            "Employer action", employer_fn(employer_view), VALID_EMPLOYER_ACTIONS
        )
        employee_action = _validate_action(
            "Employee action", employee_fn(employee_view), VALID_EMPLOYEE_ACTIONS
        )

        high_effort = int(employee_action == "high")
        low_effort = int(employee_action == "low")

        employer_payoff, employee_payoff = PAYOFF_MATRIX[(employer_action, employee_action)]

        history_records.append(
            {
                "round": round_idx,
                "employer_action": employer_action,
                "my_action": employee_action,
                "employer_payoff": employer_payoff,
                "employee_payoff": employee_payoff,
                "high_effort": high_effort,
                "low_effort": low_effort,
            }
        )

        employer_history.append(
            {
                "round": round_idx,
                "employer_action": employer_action,
                "high_effort": high_effort,
                "low_effort": low_effort,
            }
        )
        employee_history.append(
            {
                "round": round_idx,
                "employer_action": employer_action,
                "my_action": employee_action,
                "high_effort": high_effort,
                "low_effort": low_effort,
            }
        )

    history_df = pd.DataFrame(history_records)

    employer_total = int(history_df["employer_payoff"].sum())
    employee_total = int(history_df["employee_payoff"].sum())

    return MatchResult(history=history_df, employer_total=employer_total, employee_total=employee_total)


def summarize_match(result: MatchResult) -> pd.Series:
    """Aggregate totals and averages for quick inspection."""
    totals = {
        "employer_total": result.employer_total,
        "employee_total": result.employee_total,
        "rounds": len(result.history),
    }
    totals["employer_avg"] = totals["employer_total"] / totals["rounds"]
    totals["employee_avg"] = totals["employee_total"] / totals["rounds"]
    return pd.Series(totals)



In [ ]:
def run_multi_match(
    employer_fn: EmployerStrategy,
    employee_strategies: Mapping[str, EmployeeStrategy],
    rounds: int = 10,
):
    """Run one employer against many employees simultaneously with shared rounds."""
    if not employee_strategies:
        raise ValueError("employee_strategies cannot be empty")

    employee_histories: Dict[str, list[dict]] = {name: [] for name in employee_strategies}
    employer_history: list[dict] = []

    for round_idx in range(1, rounds + 1):
        employer_view = _history_frame(employer_history, EMPLOYER_HISTORY_COLUMNS)
        employer_action = _validate_action(
            "Employer action", employer_fn(employer_view), VALID_EMPLOYER_ACTIONS
        )

        employee_actions: Dict[str, EmployeeAction] = {}
        for employee_name, employee_fn in employee_strategies.items():
            history_view = _history_frame(employee_histories[employee_name], HISTORY_COLUMNS)
            employee_actions[employee_name] = _validate_action(
                f"Employee action ({employee_name})",
                employee_fn(history_view),
                VALID_EMPLOYEE_ACTIONS,
            )

        high_count = sum(action == "high" for action in employee_actions.values())
        low_count = len(employee_actions) - high_count

        for employee_name, employee_action in employee_actions.items():
            employer_payoff, employee_payoff = PAYOFF_MATRIX[(employer_action, employee_action)]
            employee_histories[employee_name].append(
                {
                    "round": round_idx,
                    "employer_action": employer_action,
                    "my_action": employee_action,
                    "high_effort": high_count,
                    "low_effort": low_count,
                    "employer_payoff": employer_payoff,
                    "employee_payoff": employee_payoff,
                }
            )

        employer_history.append(
            {
                "round": round_idx,
                "employer_action": employer_action,
                "high_effort": high_count,
                "low_effort": low_count,
            }
        )

    per_employee_results: Dict[str, MatchResult] = {}
    leaderboard_rows = []
    for employee_name, rows in employee_histories.items():
        history_df = pd.DataFrame(rows)
        employer_total = int(history_df["employer_payoff"].sum())
        employee_total = int(history_df["employee_payoff"].sum())
        result = MatchResult(history=history_df, employer_total=employer_total, employee_total=employee_total)
        per_employee_results[employee_name] = result
        leaderboard_rows.append(
            {
                "employee": employee_name,
                "rounds": len(history_df),
                **result.as_dict(),
            }
        )

    leaderboard = pd.DataFrame(leaderboard_rows)
    leaderboard["employer_avg"] = leaderboard["employer_total"] / leaderboard["rounds"]
    leaderboard["employee_avg"] = leaderboard["employee_total"] / leaderboard["rounds"]
    leaderboard = leaderboard.sort_values(by="employee_avg", ascending=False).reset_index(drop=True)

    return per_employee_results, leaderboard


def build_multi_round_tables(results: Mapping[str, MatchResult]) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Return (actions_table, scores_table) with rounds as rows and employees as columns."""
    if not results:
        raise ValueError("results cannot be empty")

    iterator = iter(results.items())
    _, first_result = next(iterator)
    round_numbers = sorted(first_result.history["round"].unique())

    action_rows: Dict[int, Dict[str, str]] = {}
    score_rows: Dict[int, Dict[str, int]] = {}

    for round_idx in round_numbers:
        first_row = first_result.history[first_result.history["round"] == round_idx].iloc[0]
        action_rows[round_idx] = {"Employer": first_row["employer_action"]}
        score_rows[round_idx] = {}
        employer_round_total = 0

        for employee_name, result in results.items():
            employee_row = result.history[result.history["round"] == round_idx].iloc[0]
            action_rows[round_idx][employee_name] = employee_row["my_action"]
            score_rows[round_idx][employee_name] = int(employee_row["employee_payoff"])
            employer_round_total += int(employee_row["employer_payoff"])

        score_rows[round_idx]["Employer"] = employer_round_total

    column_order = ["Employer"] + list(results.keys())

    action_table = pd.DataFrame.from_dict(action_rows, orient="index").sort_index()
    action_table = action_table[column_order]
    action_table.index.name = "round"

    score_table = pd.DataFrame.from_dict(score_rows, orient="index").sort_index()
    score_table = score_table[column_order]
    score_table.index.name = "round"
    score_table.loc["Total"] = score_table.sum(numeric_only=True)

    return action_table, score_table
